In [1]:
import sys
sys.path.insert(0, '../' )


In [2]:
import m_fri_ldt as fri

In [3]:
#def ldt_gen_proof( f0 , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):

###########  test   #############

proof = fri.ldt_gen_proof( [1,2,3,4,5,6,7,8] , b'123' )


rho: 8, offset: 0x8000000000000000
|original poly| : 8

### commit phase ###
Assume the evaluate on [0x8000000000000000,|8|x8) has been committed ?
|vi|: 64
do a redundent commit here for checking correctness
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [8] -> [4]:
commit evaluated valuse. --> commits: [1] <--  #mesg: 32
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
deriving new polynomial of length [4]
ibtfy_1: 0x8000000000000000 |64|
|n_vi|: 32
evaluated values generated by 1 stage of ibtfy  [32]
iteration 1: [4] -> [2]:
commit evaluated valuse. --> commits: [2] <--  #mesg: 16
update h_state <- H( h_state|| xi ): b'\x94:T\xbb\x9a\xf1W@\xb6\xee\xda\x17\x13\xa3\x7fW\xee\xde\x89\xb4\xd1m\x9bj)T\x1a\x0b\xa4\x02f\x9d'
update i: 2
derive new challenge xi <- H

In [4]:
#def ldt_verify( proof , _poly_len , h_state , Nq = 26 , RS_rho = 8 , verbose = 1 ):

############# test  ##############

fri.ldt_verify( proof , 8 , b'123' )



######## recovery hash state and challenges ########
derive x0 <- H( h_state || 3 || 1 ): 0x5b9610dcbdcfa2f0b480abae7e2526ddad75cf20f9becf1b
iteration 0: [8] -> [4]:
mt.root = commits[0] =  b'\xc3\xc7`\x12\x03;\xe7@\xb2S\xfe\xb9\xc8)\xe5\xaf\xeb\xc2\x10\xb9\x04C|\n`\xa2p\xeb\xba\xcfy\xc7'
update h_state <- H( h_state|| xi ): b'\xf8\xff?\x90]\xfe\x94?jM\n\x0cH\xba\x04nU\xd0\xed\xe5De3r<-n\x9fL\xd2R\xf6'
update i: 1
derive new challenge xi <- H(h_state||4||1) : 0xc2699f897fff266d98801881da3468244cf53838795df1ef
new polynomial length [4]
iteration 1: [4] -> [2]:
mt.root = commits[1] =  b'\x1e!\xff\xff/ \xab\x9fd\x8bS. \x9b\x1b\xbb\xadA\x10\xb0^\xa2n&\xc9n\xc1\xdf1\xe6)\xbd'
update h_state <- H( h_state|| xi ): b'\x94:T\xbb\x9a\xf1W@\xb6\xee\xda\x17\x13\xa3\x7fW\xee\xde\x89\xb4\xd1m\x9bj)T\x1a\x0b\xa4\x02f\x9d'
update i: 2
derive new challenge xi <- H(h_state||5||1) : 0x60bf68e365f0747c6446e96830a1b8df0b15c2b29ab39427
new polynomial length [2]
update h_state <- H( xi || deg1poly ): b'\xf4!

True